In [1]:
import os
import numpy as np
import signal_processing as sp
import matplotlib.pyplot as plt
import time
import sklearn
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from sys import getsizeof

In [2]:
# Import datasets
dataset_directory = os.path.join(os.getcwd(), "DeepSig-Dataset-2018")

# Use these to load the entire dataset into memory
fft_dataset_train = np.load(os.path.join(dataset_directory, "fft_dataset_train.npy"))
fft_dataset_val = np.load(os.path.join(dataset_directory, "fft_dataset_val.npy"))
fft_dataset_test = np.load(os.path.join(dataset_directory, "fft_dataset_test.npy"))

labels_train = np.load(os.path.join(dataset_directory, "labels_train.npy"))
labels_val = np.load(os.path.join(dataset_directory, "labels_val.npy"))
labels_test = np.load(os.path.join(dataset_directory, "labels_test.npy"))

# # Use these to use a memory map to save RAM
# fft_dataset_train = np.load(os.path.join(dataset_directory, "fft_dataset_train.npy"), mmap_mode="r+")
# fft_dataset_val = np.load(os.path.join(dataset_directory, "fft_dataset_val.npy"), mmap_mode="r+")
# fft_dataset_test = np.load(os.path.join(dataset_directory, "fft_dataset_test.npy"), mmap_mode="r+")

# labels_train = np.load(os.path.join(dataset_directory, "labels_train.npy"), mmap_mode="r+")
# labels_val = np.load(os.path.join(dataset_directory, "labels_val.npy"), mmap_mode="r+")
# labels_test = np.load(os.path.join(dataset_directory, "labels_test.npy"), mmap_mode="r+")

Super crude hyper parameter tuning by creating multiple different models

In [3]:
# Create list of models
models = []

In [4]:
# Large model SNR 10
inputs = keras.Input(shape=(1024, 1))
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=512, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=512, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=512, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(128)(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(24, activation='softmax')(x)

models.append(keras.Model(inputs=inputs, outputs=outputs, name = "snr_10_new_model"))

models[-1].summary()

Model: "snr_10_new_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1024, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1022, 64)          256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1020, 64)          12352     
_________________________________________________________________
batch_normalization (BatchNo (None, 1020, 64)          256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 510, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 508, 64)           12352     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 506, 64)      

In [5]:
# Create list of optimizers to use
optimizers = [keras.optimizers.RMSprop(learning_rate=0.005, momentum=0.9)]

In [6]:
# # Enable for quick testing
# fft_dataset_train = fft_dataset_train[:100]
# fft_dataset_val = fft_dataset_val[:100]
# fft_dataset_test = fft_dataset_test[:100]

# labels_train = labels_train[:100]
# labels_val = labels_val[:100]
# labels_test = labels_test[:100]

In [7]:
for optimizer in optimizers:
    for model in models:
        # Time the time it takes to train the model
        start = time.time()
        
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Define the callbacks and save the best model to a new file
        callbacks = [keras.callbacks.ModelCheckpoint(filepath=f'models/{model.name}.keras', save_best_only=True, monitor='val_loss'), 
            keras.callbacks.EarlyStopping(monitor="val_loss", min_delta = 0.002, patience = 10, verbose = 1, restore_best_weights = True)]
        # Train model
        history = model.fit(fft_dataset_train, labels_train, epochs=100, batch_size = 256, validation_data = (fft_dataset_val, labels_val), callbacks=callbacks)

        # Save pickle of history
        with open(f"models/{model.name}.pickle", mode='wb') as f:
            pickle.dump(history.history, f)

        # Evaluate model
        _, accuracy = model.evaluate(fft_dataset_test, labels_test)

        print("--- Model trained in %s seconds ---" % (time.time() - start))

        # Write accuracy to file
        f = open("models/accuracy_log.txt", "a")
        f.write(f"Model {model.name} evaluated an accuracy of {accuracy}.\n")
        f.write(f"Model {model.name} trained in {int(time.time() - start)} seconds.\n\n")
        f.close()

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 5.6222 - accuracy: 0.0600 - val_loss: 101312.5938 - val_accuracy: 0.0700
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 50.1746 - accuracy: 0.1600 - val_loss: 75303272.0000 - val_accuracy: 0.0200
Epoch 3/100
1/1 [==============================] - 1s 929ms/step - loss: 546.2011 - accuracy: 0.0300 - val_loss: 1551904768.0000 - val_accuracy: 0.0300
Epoch 4/100
1/1 [==============================] - 1s 870ms/step - loss: 117.0104 - accuracy: 0.1100 - val_loss: 40148398080.0000 - val_accuracy: 0.0700
Epoch 5/100
1/1 [==============================] - 1s 872ms/step - loss: 718.3706 - accuracy: 0.1100 - val_loss: 27937492992.0000 - val_accuracy: 0.0700
Epoch 6/100
1/1 [==============================] - 1s 900ms/step - loss: 172.0324 - accuracy: 0.0800 - val_loss: 13932405760.0000 - val_accuracy: 0.0700
Epoch 7/100
1/1 [==============================] - 1s 887ms/step - loss: 259.4644 - accuracy: 0.1100